In [1]:
using StatisticalRethinking, Mamba

# Data
line = Dict{Symbol, Any}()

howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

Use only adults

In [2]:
df2 = filter(row -> row[:age] >= 18, df);
line[:x] = convert(Array{Float64,1}, df2[:weight]);
line[:y] = convert(Array{Float64,1}, df2[:height]);
line[:xmat] = convert(Array{Float64,2}, [ones(length(line[:x])) line[:x]])

352×2 Array{Float64,2}:
 1.0  47.8256
 1.0  36.4858
 1.0  31.8648
 1.0  53.0419
 1.0  41.2769
 1.0  62.9926
 1.0  38.2435
 1.0  55.48  
 1.0  34.8699
 1.0  54.4877
 ⋮           
 1.0  47.8823
 1.0  39.4058
 1.0  41.0501
 1.0  40.8233
 1.0  47.0318
 1.0  34.2462
 1.0  52.1631
 1.0  54.0625
 1.0  52.5316

Model Specification

In [3]:
model = Model(
  y = Stochastic(1,
    (xmat, beta, s2) -> MvNormal(xmat * beta, sqrt(s2)),
    false
  ),
  beta = Stochastic(1, () -> MvNormal([178, 0], [sqrt(10000), sqrt(100)])),
  s2 = Stochastic(() -> Uniform(0, 50))
)

Object of type "Mamba.Model"
-------------------------------------------------------------------------------
beta:
A monitored node of type "0-element Mamba.ArrayStochastic{1}"
Float64[]
-------------------------------------------------------------------------------
y:
An unmonitored node of type "0-element Mamba.ArrayStochastic{1}"
Float64[]
-------------------------------------------------------------------------------
s2:
A monitored node of type "Mamba.ScalarStochastic"
NaN


Initial Values

In [4]:
inits = [
  Dict{Symbol, Any}(
    :y => line[:y],
    :beta => [rand(Normal(178, 100)), rand(Normal(0, 10))],
    :s2 => rand(Uniform(0, 50))
  )
  for i in 1:3
]

3-element Array{Dict{Symbol,Any},1}:
 Dict(:beta=>[200.628, -12.8201],:y=>[151.765, 139.7, 136.525, 156.845, 145.415, 163.83, 149.225, 168.91, 147.955, 165.1  …  156.21, 160.655, 146.05, 156.21, 152.4, 162.56, 142.875, 162.56, 156.21, 158.75],:s2=>33.7)  
 Dict(:beta=>[234.191, 4.20307],:y=>[151.765, 139.7, 136.525, 156.845, 145.415, 163.83, 149.225, 168.91, 147.955, 165.1  …  156.21, 160.655, 146.05, 156.21, 152.4, 162.56, 142.875, 162.56, 156.21, 158.75],:s2=>6.73949)
 Dict(:beta=>[75.538, -13.75],:y=>[151.765, 139.7, 136.525, 156.845, 145.415, 163.83, 149.225, 168.91, 147.955, 165.1  …  156.21, 160.655, 146.05, 156.21, 152.4, 162.56, 142.875, 162.56, 156.21, 158.75],:s2=>43.283)   

Tuning Parameters

In [5]:
scale1 = [0.5, 0.25]
summary1 = identity
eps1 = 0.5

scale2 = 0.5
summary2 = x -> [mean(x); sqrt(var(x))]
eps2 = 0.1

0.1

Define sampling scheme

In [6]:
scheme = [
  Mamba.NUTS([:beta]),
  Mamba.Slice([:s2], 10)
]

setsamplers!(model, scheme)

Object of type "Mamba.Model"
-------------------------------------------------------------------------------
beta:
A monitored node of type "0-element Mamba.ArrayStochastic{1}"
Float64[]
-------------------------------------------------------------------------------
y:
An unmonitored node of type "0-element Mamba.ArrayStochastic{1}"
Float64[]
-------------------------------------------------------------------------------
s2:
A monitored node of type "Mamba.ScalarStochastic"
NaN


MCMC Simulation

In [7]:
sim = mcmc(model, line, inits, 10000, burnin=1000, chains=3)

MCMC Simulation of 10000 Iterations x 3 Chains...

Chain 1:   0% [0:14:13 of 0:14:14 remaining]
Chain 1:  10% [0:01:08 of 0:01:16 remaining]
Chain 1:  20% [0:00:59 of 0:01:14 remaining]
Chain 1:  30% [0:00:52 of 0:01:15 remaining]
Chain 1:  40% [0:00:45 of 0:01:15 remaining]
Chain 1:  50% [0:00:37 of 0:01:14 remaining]
Chain 1:  60% [0:00:30 of 0:01:14 remaining]
Chain 1:  70% [0:00:23 of 0:01:15 remaining]
Chain 1:  80% [0:00:15 of 0:01:15 remaining]
Chain 1:  90% [0:00:07 of 0:01:15 remaining]
Chain 1: 100% [0:00:00 of 0:01:14 remaining]

Chain 2:   0% [0:00:02 of 0:00:02 remaining]
Chain 2:  10% [0:00:52 of 0:00:58 remaining]
Chain 2:  20% [0:00:57 of 0:01:12 remaining]
Chain 2:  30% [0:00:53 of 0:01:16 remaining]
Chain 2:  40% [0:00:47 of 0:01:18 remaining]
Chain 2:  50% [0:00:39 of 0:01:18 remaining]
Chain 2:  60% [0:00:32 of 0:01:19 remaining]
Chain 2:  70% [0:00:24 of 0:01:19 remaining]
Chain 2:  80% [0:00:16 of 0:01:20 remaining]
Chain 2:  90% [0:00:08 of 0:01:20 remaining]
Cha

Object of type "Mamba.ModelChains"

Iterations = 1001:10000
Thinning interval = 1
Chains = 1,2,3
Samples per chain = 9000

[26.8672 113.969 0.905587; 31.8816 113.969 0.907221; … ; 27.9261 113.77 0.90902; 26.72 115.47 0.86967]

[24.0536 115.894 0.858569; 29.1813 115.882 0.882856; … ; 26.5788 112.602 0.945593; 25.2993 111.151 0.965024]

[27.7913 114.361 0.888448; 24.2817 114.358 0.906613; … ; 23.3248 114.962 0.889329; 29.6725 114.962 0.889329]

Show draws summary

In [8]:
describe(sim)

Iterations = 1001:10000
Thinning interval = 1
Chains = 1,2,3
Samples per chain = 9000

Empirical Posterior Estimates:
            Mean         SD        Naive SE        MCSE        ESS   
     s2  26.1843337 1.998286509 0.01216118441 0.01993462054 9000.0000
beta[1] 113.9548798 1.889631383 0.01149993038 0.03566968604 2806.4390
beta[2]   0.9034166 0.041528897 0.00025273682 0.00077784588 2850.4517

Quantiles:
            2.5%       25.0%      50.0%       75.0%       97.5%  
     s2  22.5578322  24.803400  26.0794476  27.4340493  30.448849
beta[1] 110.3128920 112.700494 113.9347864 115.1774804 117.778357
beta[2]   0.8193789   0.876644   0.9037803   0.9310755   0.984011



End of `clip_38.0m.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*